In [87]:
%pip install astrapy python-dotenv pandas datasets

Note: you may need to restart the kernel to use updated packages.


In [104]:
import json
from astrapy.db import AstraDBCollection, AstraDB
import os
from dotenv import load_dotenv
import pandas as pd
from datasets import Dataset
from pprint import pprint
import random
from IPython.display import clear_output
import time

In [89]:
load_dotenv()
# AstraDB connection information
token = os.getenv("token")
endpoint = os.getenv("endpoint")

collection_name = "test_instructions"
astra_db = AstraDB(token=token, api_endpoint=endpoint)
collection = AstraDBCollection(collection_name=collection_name, astra_db=astra_db)

In [90]:
nextPageState = ""
raw_dataset = []
expected_columns = ['_id','instruction', 'input', 'output', 'original_response', 'fine_tuned_response']

def check_expected_columns(raw_instruction):
  if all(column in raw_instruction for column in expected_columns):
    return True
  else:
    return False

while nextPageState != None:
  if nextPageState == "":
    data = collection.find()
    nextPageState = data['data']['nextPageState']
    raw_instructions = [instruction for instruction in data['data']['documents'] if check_expected_columns(instruction)]
    raw_dataset.extend(raw_instructions)
  else:
    data = collection.find(options={"pageState":nextPageState}, sort = None)
    nextPageState = data['data']['nextPageState']
    raw_instructions = [instruction for instruction in data['data']['documents'] if check_expected_columns(instruction)]
    raw_dataset.extend(raw_instructions)

dataframe = pd.DataFrame(data=raw_dataset, dtype='string')
#dataframe.info()
dataset = Dataset.from_pandas(dataframe)
#pprint(dataset[0])

idx_min = 0
idx_max = 100
partial_dataset = dataset.filter(lambda example, idx: idx >= idx_min and idx < idx_max, with_indices=True)
shuffled_dataset = partial_dataset.shuffle().flatten_indices()

Flattening the indices: 100%|██████████| 100/100 [00:00<00:00, 21568.98 examples/s]


In [91]:
def split_long_string(string, width):
    if len(string) > width:
        return [string[0:width]]+split_long_string(string[width:], width)
    else:
        return [string]
        
def print_two_columns(string1, string2, width):
    i = 0
    list1 = [ split_long_string(line, width) for line in string1.split("\n") ]
    list2 = [ split_long_string(line, width) for line in string2.split("\n") ]
    lines1 = [item for row in list1 for item in row]
    lines2 = [item for row in list2 for item in row]
    max_len = max(len(lines1), len(lines2))
    table = []
    padding = 10
    col_width = width + padding
    for i in range(max_len):
        column1 = ""
        column2 = ""
        if i < len(lines1):
            column1 = lines1[i]
        if i < len(lines2):
            column2 = lines2[i]
        row = [column1, column2]
        table.append(row)
    for row in table:
        format_string = "{:>"+str(col_width)+"} {:>"+str(col_width)+"}"
        print(format_string.format(*row))



In [98]:
user_name = "oby"

In [105]:
eval_responses = []

for row in shuffled_dataset:
    order = random.randint(0,1)
    if order == 0:
        responses = ["Response #1: \n"+row['original_response'], "Response #2: \n"+row['fine_tuned_response']]
    else:
        responses = ["Response #1: \n"+row['fine_tuned_response'], "Response #2: \n"+row['original_response']]
    print("Rate each of the following responses to the input prompt out of ten.")
    print("Instruction: ")
    pprint(row['instruction'])
    print("Context:")
    pprint(row['input'])
    print("\n")
    print_two_columns(responses[0], responses[1], 70)
    time.sleep(1)
    row_result = input("Please enter your ratings as two numbers separated by a comma.")
    if "," in row_result:
        row_result = [int(rating) for rating in row_result.split(",")]
    if "." in row_result:
        row_result = [int(rating) for rating in row_result.split(".")]
    if " " in row_result:
        row_result = [int(rating) for rating in row_result.split(" ")]
    original_rating = 0
    fine_tuned_rating = 0
    if order == 0:
        original_rating = row_result[0]
        fine_tuned_rating = row_result[1]
    else:
        original_rating = row_result[1]
        fine_tuned_rating = row_result[0]

    output_line = {
        "_id": row['_id'],
        "original_rating": original_rating,
        "fine_tuned_rating": fine_tuned_rating
    }

    eval_responses.append(output_line)
    clear_output(wait=True)

with open(user_name+'_eval_out_of_ten.json', 'w', encoding='utf-8') as f:
    json.dump(eval_responses, f, ensure_ascii=False, indent=4)

    

Rate each of the following responses to the input prompt out of ten.
Instruction: 
'Explain any Caveats or Cautions noted in the Article.'
Context:
('Secondary Indexes in Cassandra can cause performance issues, especially if a '
 'query needs to access multiple nodes. They should only be used on columns or '
 'tables with low cardinality, no counters, infrequent updates, or without '
 'large partitions. Additionally, avoid using secondary indexes on high '
 'cardinality columns, tables that are frequently updated, or for ranged '
 "queries (> or <) due to limitations in Cassandra's indexing mechanism.")


                                                                   Response #1:                                                                     Response #2: 
                                                                                                                                                                 
                                                               

KeyboardInterrupt: 